# Homework #1 

In [166]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True)
%matplotlib inline

df = pd.read_fwf("./auto-mpg.data", header=None)
df = df[( df[3]!='?' )]
df.columns = ['mpg', 'cylinders', 'displacement', 'hp', 'weight', 'acceleration', 'year','origin', 'name']

## Problem 1

In [169]:
groups = np.array_split(df.sort_values(by="mpg"), 4)
groups[0]['color'] = "green"
groups[1]['color'] = "yellow"
groups[2]['color'] = "red"
groups[3]['color'] = "black"
df = pd.concat(groups)

## Problem 2

In [3]:
# sns.pairplot(df, hue="color", diag_kind="hist", palette="husl", vars=['cylinders', 'displacement', 'hp', 'weight', 'acceleration', 'year','origin'])
# s2=sns.pairplot(df, vars=['cylinders', 'displacement', 'hp', 'weight', 'acceleration', 'year','origin'])


## Problem 3

In [209]:
# from sklearn.datasets import load_boston
# from statsmodels.regression.linear_model import OLS
from numpy.linalg import inv

def lin_reg_single_var( x, n, y=df['mpg'] ):
    if len(x) != len(y):
        return "The dimensions of X and Y don't match!"
    
    X = None
    Y = np.array([y])
    ones = np.ones(shape=y.shape)[..., None]
    base = np.array([x]).astype(np.float).transpose() #turn "hp" into char. why?

    if n == 0:
        X = ones
    else:
        cols = base
        for i in range(2, n+1):
            cols = np.concatenate((cols, base**i), 1)
        X = np.concatenate((ones, cols), 1)

    return inv(np.transpose(X).dot(X)).dot(np.transpose(X)).dot(y)

## Problem 4

In [156]:
from sklearn.model_selection import train_test_split
x_data = df.drop(columns=["mpg","name", "color"])
y_data = df["mpg"]
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=100, random_state=42, shuffle=1)

In [273]:
def feature_regress():
    w = []
    for feature in X_train:
        row = []
        for i in range(0,4):
            row.append( lin_reg_single_var(X_train[feature], i, y_train).round(2) )

            
            lin_reg_single_var(X_train[feature], i, y_train).round(2)
            
            
            
        w.append(row)
    return w
feature_regress()

[[array([23.76]),
  array([42.87, -3.55]),
  array([48.76, -5.74,  0.19]),
  array([-127.55,   92.17,  -17.18,    0.99])],
 [array([23.76]),
  array([35.16, -0.06]),
  array([41.76, -0.14,  0.  ]),
  array([43.35, -0.16,  0.  , -0.  ])],
 [array([23.76]),
  array([40.58, -0.16]),
  array([57.33, -0.47,  0.  ]),
  array([65.36, -0.69,  0.  , -0.  ])],
 [array([23.76]),
  array([ 4.643e+01, -1.000e-02]),
  array([ 6.274e+01, -2.000e-02,  0.000e+00]),
  array([ 6.379e+01, -2.000e-02,  0.000e+00, -0.000e+00])],
 [array([23.76]),
  array([5.75, 1.16]),
  array([-32.7 ,   6.11,  -0.15]),
  array([-37.05,   6.96,  -0.21,   0.  ])],
 [array([23.76]),
  array([-64.12,   1.15]),
  array([ 6.2504e+02, -1.6990e+01,  1.2000e-01]),
  array([ 4.54014e+03, -1.71860e+02,  2.16000e+00, -1.00000e-02])],
 [array([23.76]),
  array([14.91,  5.45]),
  array([ 9.28, 12.73, -1.87]),
  array([  4.47,  60.98, -19.33,   2.69])]]